In [8]:
from llm_execution_time_predictor.train_utils import (
    build_stage_features,
    train_linear_predictor,
    train_tree_predictor,
)
import pandas as pd
import json

In [2]:
import pandas as pd

# List your files here
files = [
    "prefill_profile_profiling_tp0.jsonl",
    "profile_decode_profiling_tp0.jsonl",
    "test_profile_results.jsonl",
]
dfs = [pd.read_json(f, lines=True) for f in files]
combined_df = pd.concat(dfs, ignore_index=True)
combined_df.drop(["throughput", "timestamp", "process_id"], axis=1, inplace=True)

In [3]:
# combined_df.to_csv("combined_profile_results.csv", index=False)
import numpy as np

combined_df["input_len"] = combined_df["batch_lens"].apply(
    lambda x: np.mean(x) if isinstance(x, list) else 0
)
prefill_df = combined_df[combined_df["forward_mode"] == "prefill"]
decode_df = combined_df[combined_df["forward_mode"] == "decode"]

In [4]:
# convert the 'batch_lens' column into a plain Python list

In [5]:
train_df_prefill = build_stage_features(prefill_df, stage="prefill")
train_df_decode = build_stage_features(decode_df, stage="decode")

In [9]:
lr_model_prefill = train_linear_predictor(train_df_prefill, "prefill")
lr_model_decode = train_linear_predictor(train_df_decode, "decode")

Linear Regression: prefill
Train RMSE: 8.32ms
Train MAE: 5.03ms
Train R2: 0.9962
Linear Regression: decode
Train RMSE: 3.17ms
Train MAE: 1.93ms
Train R2: 0.9354


In [10]:
tr_model_prefill = train_tree_predictor(train_df_prefill, "prefill")
tr_model_decode = train_tree_predictor(train_df_decode, "decode")

Decision Tree: prefill
Train RMSE: 7.82ms
Train MAE: 3.61ms
Train R2: 0.9967
Decision Tree: decode
Train RMSE: 2.09ms
Train MAE: 0.17ms
Train R2: 0.9720
